With classes

In [4]:

import numpy as np

class DecisionTree:
    def __init__(self, max_depth=10, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

    def fit(self, X, y):
        self.n_features_ = X.shape[1]
        self.tree_ = self.build_tree(X, y)

    def build_tree(self, X, y, depth=0):
        n_samples = X.shape[0]
        n_labels = len(np.unique(y))

        # Stop conditions:
        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = self.get_leaf_value(y)
            return {'leaf_value': leaf_value}

        # Find the best feature to split on:
        feature_idxs = np.random.choice(self.n_features_, int(np.sqrt(self.n_features_)), replace=False)
        best_feature, best_threshold = self.get_best_feature(X, y, feature_idxs)

        # Split data based on best feature and threshold:
        left_idxs = X[:, best_feature] < best_threshold
        right_idxs = X[:, best_feature] >= best_threshold
        left = self.build_tree(X[left_idxs], y[left_idxs], depth+1)
        right = self.build_tree(X[right_idxs], y[right_idxs], depth+1)

        return {'feature_idx': best_feature, 'threshold': best_threshold,
                'left': left, 'right': right}

    def get_best_feature(self, X, y, feature_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feature_idx in feature_idxs:
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                gain = self.get_gain(X, y, feature_idx, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature_idx
                    split_threshold = threshold

        return split_idx, split_threshold

    def get_gain(self, X, y, feature_idx, threshold):
        left_idxs = X[:, feature_idx] < threshold
        right_idxs = X[:, feature_idx] >= threshold

        n_left, n_right = len(y[left_idxs]), len(y[right_idxs])
        n_total = n_left + n_right

        if n_left == 0 or n_right == 0:
            return 0

        gain = self.entropy(y) - (n_left/n_total)*self.entropy(y[left_idxs]) - (n_right/n_total)*self.entropy(y[right_idxs])

        return gain

    def get_leaf_value(self, y):
        unique, counts = np.unique(y, return_counts=True)
        most_common_label = unique[np.argmax(counts)]
        return most_common_label

    def predict(self, X):
        return np.array([self.traverse_tree(x, self.tree_) for x in X])

    def traverse_tree(self, x, node):
        if 'leaf_value' in node:
            return node['leaf_value']

        if x[node['feature_idx']] < node['threshold']:
            return self.traverse_tree(x, node['left'])
        else:
            return self.traverse_tree(x, node['right'])

    def entropy(self, y):
        unique, counts = np.unique(y, return_counts=True)
        p = counts / len(y)
        return -np.sum(p * np.log2(p))


In [5]:
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.metrics import accuracy_score

# Load the breast cancer dataset
X_bc, y_bc = load_breast_cancer(return_X_y=True)

# Split the dataset into training and testing sets
X_train_bc, X_test_bc, y_train_bc, y_test_bc = train_test_split(X_bc, y_bc, test_size=0.2, random_state=42)

# Fit the decision tree model
dt_bc = DecisionTree(max_depth=5)
dt_bc.fit(X_train_bc, y_train_bc)

# Make predictions on the testing set
y_pred_bc = dt_bc.predict(X_test_bc)

# Calculate accuracy of the model on the testing set
acc_bc = accuracy_score(y_test_bc, y_pred_bc)
print(f"Breast cancer dataset accuracy: {acc_bc:.3f}")

# Load the diabetes dataset
X_db, y_db = load_diabetes(return_X_y=True)

# Split the dataset into training and testing sets
X_train_db, X_test_db, y_train_db, y_test_db = train_test_split(X_db, y_db, test_size=0.2, random_state=42)

# Fit the decision tree model
dt_db = DecisionTree(max_depth=5)
dt_db.fit(X_train_db, y_train_db)

# Make predictions on the testing set
y_pred_db = dt_db.predict(X_test_db)

# Calculate mean squared error of the model on the testing set
mse_db = mean_squared_error(y_test_db, y_pred_db)
print(f"Diabetes dataset MSE: {mse_db:.3f}")

# Calculate root mean squared error of the model on the testing set
rmse_db = np.sqrt(mse_db)
print(f"Diabetes dataset RMSE: {rmse_db:.3f}")

# Calculate mean absolute error of the model on the testing set
mae_db = mean_absolute_error(y_test_db, y_pred_db)
print(f"Diabetes dataset MAE: {mae_db:.3f}")


Breast cancer dataset accuracy: 0.947
Diabetes dataset MSE: 7942.416
Diabetes dataset RMSE: 89.120
Diabetes dataset MAE: 69.360


Regression

In [6]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

# Define the mean squared error (MSE) function
def mse(y):
    return np.mean((y - np.mean(y))**2)

# Define a function to split the data into left and right subsets based on a threshold
def split(X, y, feature_idx, threshold):
    left_indices = np.where(X[:, feature_idx] <= threshold)[0]
    right_indices = np.where(X[:, feature_idx] > threshold)[0]
    if len(left_indices) == 0 or len(right_indices) == 0:
        return None, None, None, None
    else:
        return X[left_indices], y[left_indices], X[right_indices], y[right_indices]

# Define a function to find the best split based on minimizing the MSE
def best_split(X, y):
    best_feature_idx, best_threshold, best_mse = None, None, np.inf
    # Loop through all the features in X
    for feature_idx in range(X.shape[1]):
        # Find all the unique values of the feature
        thresholds = np.unique(X[:, feature_idx])
        # Loop through all the unique values of the feature
        for threshold in thresholds:
            # Split the data into left and right subsets based on the threshold
            X_left, y_left, X_right, y_right = split(X, y, feature_idx, threshold)
            # Check that the split is valid (i.e. neither the left nor right subset is empty)
            if y_left is not None and y_right is not None:
                # Calculate the total MSE of the left and right subsets
                total_mse = mse(y_left) + mse(y_right)
                # Update the best split if the total MSE is lower than the current best MSE
                if total_mse < best_mse:
                    best_feature_idx, best_threshold, best_mse = feature_idx, threshold, total_mse
    return best_feature_idx, best_threshold

# Define a function to build the decision tree recursively
def build_tree(X, y, depth, max_depth):
    # Check if the maximum depth has been reached or if there is no further reduction in MSE
    if depth == max_depth or mse(y) == 0:
        # Return the mean target value
        return np.mean(y)
    # Find the best split based on minimizing the MSE
    feature_idx, threshold = best_split(X, y)
    # Check if there is no valid split
    if feature_idx is None:
        # Return the mean target value
        return np.mean(y)
    # Split the data into left and right subsets based on the best split
    else:
        X_left, y_left, X_right, y_right = split(X, y, feature_idx, threshold)
        # Recursively build the left and right subtrees
        left_tree = build_tree(X_left, y_left, depth+1, max_depth)
        right_tree = build_tree(X_right, y_right, depth+1, max_depth)
        # Return the decision node with the best split and the left and right subtrees
        return (feature_idx, threshold, left_tree, right_tree)

# Define a function to make predictions for a single input using the decision tree


def predict_one(x, tree):
    # Check if the current node is a leaf node (i.e. a float value)
    if isinstance(tree, float):
        # Return the mean target value of the leaf node
        return tree
    
    # Check which subtree to go down based on the feature value of the current data point
    else:
        feature_idx, threshold, left_tree, right_tree = tree
        if x[feature_idx] <= threshold:
            # Recursively go down the left subtree
            return predict_one(x, left_tree)
        else:
            # Recursively go down the right subtree
            return predict_one(x, right_tree)


# A function to predict the target values of multiple data points using a decision tree
def predict(X, tree):
    # Predict the target value of each data point using the predict_one function
    return np.array([predict_one(x, tree) for x in X])


# A function to build a decision tree for regression
def decision_tree_regression(X_train, y_train, max_depth):
    # Build the decision tree using the build_tree function
    tree = build_tree(X_train, y_train, 0, max_depth)
    # Predict the target values of the training set using the predict function
    y_pred = predict(X_train, tree)
    # Return the decision tree and the predicted target values of the training set
    return tree, y_pred


# Load the diabetes dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the decision tree regression model
max_depth = 5
tree, y_pred_train = decision_tree_regression(X_train, y_train, max_depth)

# Make predictions on the testing set
y_pred_test = predict(X_test, tree)

# Print the mean squared error on the testing set
print("Mean squared error on the testing set:", mse(y_test - y_pred_test))

Mean squared error on the testing set: 5978.60270528184
